In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import yfinance as yf

# Function to scrape ticker symbols from Wikipedia for S&P 500
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find_all('td')[0].text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to scrape ticker symbols from FTSE 100
def get_ftse100_tickers():
    url = 'https://en.wikipedia.org/wiki/FTSE_100_Index'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find_all('table', {'class': 'wikitable sortable'})[1]
    tickers = []
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) > 1:
            ticker = cols[1].text.strip()
            tickers.append(ticker + '.L')  # Adding '.L' for LSE tickers
    return tickers

# Function to get ticker symbols for Euro Stoxx 600 with manual mapping
def get_euro_stoxx600_tickers():
    euro_stoxx_mapping = {
        'ASTRAZENECA': 'AZN.L',
        'NESTLE': 'NESN.SW',
        'NOVARTIS': 'NOVN.SW',
        'SHELL': 'SHEL.L',
        'SAP': 'SAP.DE',
        # Add more mappings here
    }

    url = 'https://www.stoxx.com/index-details?symbol=SXXP'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table')
    tickers = []
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) > 1:
            company_name = cols[0].text.strip().upper()
            ticker = euro_stoxx_mapping.get(company_name, None)
            if ticker:
                tickers.append(ticker)
    return tickers

# Validate ticker symbols
def validate_tickers(tickers):
    valid_tickers = []
    for ticker in tickers:
        if ticker.isalnum() or ('.L' in ticker and ticker.replace('.L', '').isalnum()):
            valid_tickers.append(ticker)
    return valid_tickers

# Function to fetch financial data using Yahoo Finance
def get_financial_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        'Ticker': ticker,
        'PE_Ratio': info.get('forwardPE', None),
        'PB_Ratio': info.get('priceToBook', None),
        'ROE': info.get('returnOnEquity', None),
        'Debt_to_Equity': info.get('debtToEquity', None),
        'Free_Cash_Flow': info.get('freeCashflow', None),
        'ROIC': info.get('returnOnAssets', None),  # ROIC might not be directly available
        'PS_Ratio': info.get('priceToSalesTrailing12Months', None),
        'PC_Ratio': info.get('priceToCashflow', None),
        'Enterprise_Value': info.get('enterpriseValue', None),
        'EV_to_EBITDA': info.get('enterpriseToEbitda', None)
    }

# Calculating financial ratios
def calculate_ratios(sp500_tickers, ftse100_tickers, euro_stoxx600_tickers):
    ratios = []

    for ticker in sp500_tickers:
        try:
            data = get_financial_data(ticker)
            ratios.append(data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    for ticker in ftse100_tickers:
        try:
            data = get_financial_data(ticker)
            ratios.append(data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    for ticker in euro_stoxx600_tickers:
        try:
            data = get_financial_data(ticker)
            ratios.append(data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return pd.DataFrame(ratios)

# Get ticker lists
sp500_tickers = get_sp500_tickers()
ftse100_tickers = get_ftse100_tickers()
euro_stoxx600_tickers = get_euro_stoxx600_tickers()

# Validate tickers
sp500_tickers = validate_tickers(sp500_tickers)
ftse100_tickers = validate_tickers(ftse100_tickers)
euro_stoxx600_tickers = validate_tickers(euro_stoxx600_tickers)

# Print number of tickers fetched and first few tickers for verification
print(f"S&P 500 Tickers ({len(sp500_tickers)}): {sp500_tickers[:5]} ...")
print(f"FTSE 100 Tickers ({len(ftse100_tickers)}): {ftse100_tickers[:5]} ...")
print(f"Euro Stoxx 600 Tickers ({len(euro_stoxx600_tickers)}): {euro_stoxx600_tickers[:5]} ...")

# Calculate financial ratios
ratios = calculate_ratios(sp500_tickers, ftse100_tickers, euro_stoxx600_tickers)

# Print number of tickers processed and first few tickers for verification
print(f"Total Tickers Processed: {len(ratios)}")
print(ratios.head())

# Check if any FTSE 100 and Euro Stoxx 600 tickers are present in the final DataFrame
ftse_in_ratios = [ticker for ticker in ftse100_tickers if ticker in ratios['Ticker'].values]
euro_stoxx_in_ratios = [ticker for ticker in euro_stoxx600_tickers if ticker in ratios['Ticker'].values]
print(f"FTSE 100 Tickers in Final DataFrame: {len(ftse_in_ratios)}")
print(ftse_in_ratios)
print(f"Euro Stoxx 600 Tickers in Final DataFrame: {len(euro_stoxx_in_ratios)}")
print(euro_stoxx_in_ratios)

# Display all companies without applying any filters
pd.set_option('display.max_columns', None)  # Ensure all columns are shown
pd.set_option('display.width', None)        # Ensure the output fits the width of the terminal
print(ratios.to_string(index=False))

S&P 500 Tickers (501): ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN'] ...
FTSE 100 Tickers (99): ['III.L', 'ADM.L', 'AAF.L', 'AAL.L', 'ANTO.L'] ...
Euro Stoxx 600 Tickers (2): ['AZN.L', 'SHEL.L'] ...
Total Tickers Processed: 602
  Ticker   PE_Ratio   PB_Ratio      ROE  Debt_to_Equity  Free_Cash_Flow     ROIC  PS_Ratio  \
0    MMM  15.985970  17.582773  0.24314         345.812    1.080262e+10  0.07516  2.111073   
1    AOS  18.516129   6.131075  0.30342           8.637    3.881000e+08  0.14533  2.983202   
2    ABT  21.376460   4.862156  0.14441          37.525    5.387750e+09  0.06255  4.694205   
3   ABBV  15.687086  41.786110      NaN         924.059             NaN      NaN  6.085844   
4    ACN  24.742363   7.136146  0.26000          16.078    8.045156e+09  0.12244  3.069557   

  PC_Ratio  Enterprise_Value  EV_to_EBITDA  
0     None      7.100210e+10         9.660  
1     None      1.163881e+10        14.041  
2     None      1.991524e+11        18.840  
3     None      3.874479e+11        

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import yfinance as yf

# Function to scrape ticker symbols from Wikipedia for S&P 500
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find_all('td')[0].text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to scrape ticker symbols from FTSE 100
def get_ftse100_tickers():
    url = 'https://en.wikipedia.org/wiki/FTSE_100_Index'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find_all('table', {'class': 'wikitable sortable'})[1]
    tickers = []
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) > 1:
            ticker = cols[1].text.strip()
            tickers.append(ticker + '.L')  # Adding '.L' for LSE tickers
    return tickers

# Function to get ticker symbols for Euro Stoxx 600
def get_euro_stoxx600_tickers():
    url = 'https://www.stoxx.com/index-details?symbol=SXXP'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table')
    tickers = []
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) > 1:
            ticker = None
            for col in cols:
                text = col.text.strip()
                if text.isupper() and len(text) <= 5:  # Assuming ticker symbols are upper case and short
                    ticker = text
                    break
            if ticker:
                tickers.append(ticker)
    return tickers

# Validate ticker symbols
def validate_tickers(tickers):
    valid_tickers = []
    for ticker in tickers:
        if ticker.isalnum() or ('.L' in ticker and ticker.replace('.L', '').isalnum()):
            valid_tickers.append(ticker)
    return valid_tickers

# Function to fetch financial data using Yahoo Finance
def get_financial_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        'Ticker': ticker,
        'PE_Ratio': info.get('forwardPE', None),
        'PB_Ratio': info.get('priceToBook', None),
        'ROE': info.get('returnOnEquity', None),
        'Debt_to_Equity': info.get('debtToEquity', None),
        'Free_Cash_Flow': info.get('freeCashflow', None),
        'ROIC': info.get('returnOnAssets', None),  # ROIC might not be directly available
        'PS_Ratio': info.get('priceToSalesTrailing12Months', None),
        'PC_Ratio': info.get('priceToCashflow', None),
        'Enterprise_Value': info.get('enterpriseValue', None),
        'EV_to_EBITDA': info.get('enterpriseToEbitda', None)
    }

# Calculating financial ratios
def calculate_ratios(sp500_tickers, ftse100_tickers, euro_stoxx600_tickers):
    ratios = []

    for ticker in sp500_tickers:
        try:
            data = get_financial_data(ticker)
            ratios.append(data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    for ticker in ftse100_tickers:
        try:
            data = get_financial_data(ticker)
            ratios.append(data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    for ticker in euro_stoxx600_tickers:
        try:
            data = get_financial_data(ticker)
            ratios.append(data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return pd.DataFrame(ratios)

# Get ticker lists
sp500_tickers = get_sp500_tickers()
ftse100_tickers = get_ftse100_tickers()
euro_stoxx600_tickers = get_euro_stoxx600_tickers()

# Validate tickers
sp500_tickers = validate_tickers(sp500_tickers)
ftse100_tickers = validate_tickers(ftse100_tickers)
euro_stoxx600_tickers = validate_tickers(euro_stoxx600_tickers)

# Print number of tickers fetched and first few tickers for verification
print(f"S&P 500 Tickers ({len(sp500_tickers)}): {sp500_tickers[:5]} ...")
print(f"FTSE 100 Tickers ({len(ftse100_tickers)}): {ftse100_tickers[:5]} ...")
print(f"Euro Stoxx 600 Tickers ({len(euro_stoxx600_tickers)}): {euro_stoxx600_tickers[:5]} ...")

# Calculate financial ratios
ratios = calculate_ratios(sp500_tickers, ftse100_tickers, euro_stoxx600_tickers)

# Print number of tickers processed and first few tickers for verification
print(f"Total Tickers Processed: {len(ratios)}")
print(ratios.head())

# Check if any FTSE 100 and Euro Stoxx 600 tickers are present in the final DataFrame
ftse_in_ratios = [ticker for ticker in ftse100_tickers if ticker in ratios['Ticker'].values]
euro_stoxx_in_ratios = [ticker for ticker in euro_stoxx600_tickers if ticker in ratios['Ticker'].values]
print(f"FTSE 100 Tickers in Final DataFrame: {len(ftse_in_ratios)}")
print(ftse_in_ratios)
print(f"Euro Stoxx 600 Tickers in Final DataFrame: {len(euro_stoxx_in_ratios)}")
print(euro_stoxx_in_ratios)

# Display all companies without applying any filters
pd.set_option('display.max_columns', None)  # Ensure all columns are shown
pd.set_option('display.width', None)        # Ensure the output fits the width of the terminal
print(ratios.to_string(index=False))

S&P 500 Tickers (501): ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN'] ...
FTSE 100 Tickers (99): ['III.L', 'ADM.L', 'AAF.L', 'AAL.L', 'ANTO.L'] ...
Euro Stoxx 600 Tickers (9): ['NL', 'CH', 'GB', 'CH', 'SHELL'] ...


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SHELL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SHELL&crumb=e9CL4tIZE9S


Total Tickers Processed: 609
  Ticker   PE_Ratio   PB_Ratio      ROE  Debt_to_Equity  Free_Cash_Flow     ROIC  PS_Ratio  \
0    MMM  15.987244  17.584175  0.24314         345.812    1.080262e+10  0.07516  2.111241   
1    AOS  18.526497   6.134508  0.30342           8.637    3.881000e+08  0.14533  2.984872   
2    ABT  21.392025   4.865696  0.14441          37.525    5.387750e+09  0.06255  4.697623   
3   ABBV  15.710265  41.847850      NaN         924.059             NaN      NaN  6.094836   
4    ACN  24.746672   7.137388  0.26000          16.078    8.045156e+09  0.12244  3.070092   

  PC_Ratio  Enterprise_Value  EV_to_EBITDA  
0     None      7.100210e+10         9.660  
1     None      1.163881e+10        14.041  
2     None      1.991524e+11        18.840  
3     None      3.874479e+11        14.760  
4     None      1.991375e+11        17.654  
FTSE 100 Tickers in Final DataFrame: 99
['III.L', 'ADM.L', 'AAF.L', 'AAL.L', 'ANTO.L', 'AHT.L', 'ABF.L', 'AZN.L', 'AUTO.L', 'AV.L', 'BME

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import yfinance as yf

# Function to scrape ticker symbols from Wikipedia for S&P 500
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find_all('td')[0].text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to scrape ticker symbols from FTSE 100
def get_ftse100_tickers():
    url = 'https://en.wikipedia.org/wiki/FTSE_100_Index'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find_all('table', {'class': 'wikitable sortable'})[1]
    tickers = []
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) > 1:
            ticker = cols[1].text.strip()
            tickers.append(ticker + '.L')  # Adding '.L' for LSE tickers
    return tickers

# Function to get ticker symbols for Euro Stoxx 600
def get_euro_stoxx600_tickers():
    url = 'https://www.stoxx.com/index-details?symbol=SXXP'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table')
    tickers = []
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) > 1:
            ticker = None
            for col in cols:
                text = col.text.strip()
                if text.isupper() and len(text) <= 5:  # Assuming ticker symbols are upper case and short
                    ticker = text
                    break
            if ticker:
                tickers.append(ticker)
    return tickers

# Validate ticker symbols
def validate_tickers(tickers):
    valid_tickers = []
    for ticker in tickers:
        if ticker.isalnum() or ('.L' in ticker and ticker.replace('.L', '').isalnum()):
            valid_tickers.append(ticker)
    return valid_tickers

# Function to fetch financial data using Yahoo Finance
def get_financial_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        'Ticker': ticker,
        'PE_Ratio': info.get('forwardPE', None),
        'PB_Ratio': info.get('priceToBook', None),
        'ROE': info.get('returnOnEquity', None),
        'Debt_to_Equity': info.get('debtToEquity', None),
        'Free_Cash_Flow': info.get('freeCashflow', None),
        'ROIC': info.get('returnOnAssets', None),  # ROIC might not be directly available
        'PS_Ratio': info.get('priceToSalesTrailing12Months', None),
        'PC_Ratio': info.get('priceToCashflow', None),
        'Enterprise_Value': info.get('enterpriseValue', None),
        'EV_to_EBITDA': info.get('enterpriseToEbitda', None)
    }

# Calculating financial ratios
def calculate_ratios(sp500_tickers, ftse100_tickers, euro_stoxx600_tickers):
    ratios = []

    for ticker in sp500_tickers:
        try:
            data = get_financial_data(ticker)
            ratios.append(data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    for ticker in ftse100_tickers:
        try:
            data = get_financial_data(ticker)
            ratios.append(data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    for ticker in euro_stoxx600_tickers:
        try:
            data = get_financial_data(ticker)
            ratios.append(data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return pd.DataFrame(ratios)

# Function to filter companies based on financial ratios
def filter_companies(df, pe_ratio_range=None, pb_ratio_range=None, roe_range=None, debt_to_equity_range=None,
                     free_cash_flow_range=None, roic_range=None, ps_ratio_range=None, pc_ratio_range=None,
                     ev_to_ebitda_range=None):
    filtered_df = df.copy()

    if pe_ratio_range:
        filtered_df = filtered_df[filtered_df['PE_Ratio'].between(pe_ratio_range[0], pe_ratio_range[1], inclusive='both')]
    if pb_ratio_range:
        filtered_df = filtered_df[filtered_df['PB_Ratio'].between(pb_ratio_range[0], pb_ratio_range[1], inclusive='both')]
    if roe_range:
        filtered_df = filtered_df[filtered_df['ROE'].between(roe_range[0], roe_range[1], inclusive='both')]
    if debt_to_equity_range:
        filtered_df = filtered_df[filtered_df['Debt_to_Equity'].between(debt_to_equity_range[0], debt_to_equity_range[1], inclusive='both')]
    if free_cash_flow_range:
        filtered_df = filtered_df[filtered_df['Free_Cash_Flow'].between(free_cash_flow_range[0], free_cash_flow_range[1], inclusive='both')]
    if roic_range:
        filtered_df = filtered_df[filtered_df['ROIC'].between(roic_range[0], roic_range[1], inclusive='both')]
    if ps_ratio_range:
        filtered_df = filtered_df[filtered_df['PS_Ratio'].between(ps_ratio_range[0], ps_ratio_range[1], inclusive='both')]
    if pc_ratio_range:
        filtered_df = filtered_df[filtered_df['PC_Ratio'].between(pc_ratio_range[0], pc_ratio_range[1], inclusive='both')]
    if ev_to_ebitda_range:
        filtered_df = filtered_df[filtered_df['EV_to_EBITDA'].between(ev_to_ebitda_range[0], ev_to_ebitda_range[1], inclusive='both')]

    return filtered_df

# Get ticker lists
sp500_tickers = get_sp500_tickers()
ftse100_tickers = get_ftse100_tickers()
euro_stoxx600_tickers = get_euro_stoxx600_tickers()

# Validate tickers
sp500_tickers = validate_tickers(sp500_tickers)
ftse100_tickers = validate_tickers(ftse100_tickers)
euro_stoxx600_tickers = validate_tickers(euro_stoxx600_tickers)

# Print number of tickers fetched and first few tickers for verification
print(f"S&P 500 Tickers ({len(sp500_tickers)}): {sp500_tickers[:5]} ...")
print(f"FTSE 100 Tickers ({len(ftse100_tickers)}): {ftse100_tickers[:5]} ...")
print(f"Euro Stoxx 600 Tickers ({len(euro_stoxx600_tickers)}): {euro_stoxx600_tickers[:5]} ...")

# Calculate financial ratios
ratios = calculate_ratios(sp500_tickers, ftse100_tickers, euro_stoxx600_tickers)

# Print number of tickers processed and first few tickers for verification
print(f"Total Tickers Processed: {len(ratios)}")
print(ratios.head())

# Example of filtering companies with PE Ratio between 10 and 20
filtered_companies = filter_companies(ratios, pe_ratio_range=(18, 20))

# Display filtered companies
print("Filtered Companies based on PE Ratio between 10 and 20:")
print(filtered_companies)

# Check if any FTSE 100 and Euro Stoxx 600 tickers are present in the final DataFrame
ftse_in_ratios = [ticker for ticker in ftse100_tickers if ticker in ratios['Ticker'].values]
euro_stoxx_in_ratios = [ticker for ticker in euro_stoxx600_tickers if ticker in ratios['Ticker'].values]
print(f"FTSE 100 Tickers in Final DataFrame: {len(ftse_in_ratios)}")
print(ftse_in_ratios)
print(f"Euro Stoxx 600 Tickers in Final DataFrame: {len(euro_stoxx_in_ratios)}")
print(euro_stoxx_in_ratios)

# Display all companies without applying any filters
pd.set_option('display.max_columns', None)  # Ensure all columns are shown
pd.set_option('display.width', None)        # Ensure the output fits the width of the terminal
print(ratios.to_string(index=False))

S&P 500 Tickers (501): ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN'] ...
FTSE 100 Tickers (99): ['III.L', 'ADM.L', 'AAF.L', 'AAL.L', 'ANTO.L'] ...
Euro Stoxx 600 Tickers (9): ['NL', 'CH', 'GB', 'CH', 'SHELL'] ...


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SHELL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SHELL&crumb=e9CL4tIZE9S


Total Tickers Processed: 609
  Ticker   PE_Ratio   PB_Ratio      ROE  Debt_to_Equity  Free_Cash_Flow     ROIC  PS_Ratio  \
0    MMM  15.978010  17.574018  0.24314         345.812    1.080262e+10  0.07516  2.110022   
1    AOS  18.589860   6.155489  0.30342           8.637    3.881000e+08  0.14533  2.995081   
2    ABT  21.387160   4.864590  0.14441          37.525    5.387750e+09  0.06255  4.696555   
3   ABBV  15.752069  41.959210      NaN         924.059             NaN      NaN  6.111054   
4    ACN  24.819395   7.158363  0.26000          16.078    8.045156e+09  0.12244  3.079114   

  PC_Ratio  Enterprise_Value  EV_to_EBITDA  
0     None      7.100210e+10         9.660  
1     None      1.163881e+10        14.041  
2     None      1.991524e+11        18.840  
3     None      3.874479e+11        14.760  
4     None      1.991375e+11        17.654  
Filtered Companies based on PE Ratio between 10 and 20:
    Ticker   PE_Ratio     PB_Ratio      ROE  Debt_to_Equity  Free_Cash_Flow     

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import yfinance as yf

# Function to scrape ticker symbols from Wikipedia for S&P 500
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find_all('td')[0].text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to scrape ticker symbols from FTSE 100
def get_ftse100_tickers():
    url = 'https://en.wikipedia.org/wiki/FTSE_100_Index'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find_all('table', {'class': 'wikitable sortable'})[1]
    tickers = []
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) > 1:
            ticker = cols[1].text.strip()
            tickers.append(ticker + '.L')  # Adding '.L' for LSE tickers
    return tickers

# Function to get ticker symbols for Euro Stoxx 600
def get_euro_stoxx600_tickers():
    url = 'https://www.stoxx.com/index-details?symbol=SXXP'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table')
    tickers = []
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) > 1:
            ticker = None
            for col in cols:
                text = col.text.strip()
                if text.isupper() and len(text) <= 5:  # Assuming ticker symbols are upper case and short
                    ticker = text
                    break
            if ticker:
                tickers.append(ticker)
    return tickers

# Validate ticker symbols
def validate_tickers(tickers):
    valid_tickers = []
    for ticker in tickers:
        if ticker.isalnum() or ('.L' in ticker and ticker.replace('.L', '').isalnum()):
            valid_tickers.append(ticker)
    return valid_tickers

# Function to fetch financial data using Yahoo Finance
def get_financial_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        'Ticker': ticker,
        'PE_Ratio': info.get('forwardPE', None),
        'PB_Ratio': info.get('priceToBook', None),
        'ROE': info.get('returnOnEquity', None),
        'Debt_to_Equity': info.get('debtToEquity', None),
        'Free_Cash_Flow': info.get('freeCashflow', None),
        'ROIC': info.get('returnOnAssets', None),  # ROIC might not be directly available
        'PS_Ratio': info.get('priceToSalesTrailing12Months', None),
        'Enterprise_Value': info.get('enterpriseValue', None),
        'EV_to_EBITDA': info.get('enterpriseToEbitda', None)
    }

# Calculating financial ratios
def calculate_ratios(sp500_tickers, ftse100_tickers, euro_stoxx600_tickers):
    ratios = []

    for ticker in sp500_tickers:
        try:
            data = get_financial_data(ticker)
            ratios.append(data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    for ticker in ftse100_tickers:
        try:
            data = get_financial_data(ticker)
            ratios.append(data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    for ticker in euro_stoxx600_tickers:
        try:
            data = get_financial_data(ticker)
            ratios.append(data)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return pd.DataFrame(ratios)

# Function to filter companies based on financial ratios
def filter_companies(df, pe_ratio_range=None, pb_ratio_range=None, roe_range=None, debt_to_equity_range=None,
                     free_cash_flow_range=None, roic_range=None, ps_ratio_range=None, ev_to_ebitda_range=None):
    filtered_df = df.copy()

    if pe_ratio_range:
        filtered_df = filtered_df[filtered_df['PE_Ratio'].between(pe_ratio_range[0], pe_ratio_range[1], inclusive='both')]
    if pb_ratio_range:
        filtered_df = filtered_df[filtered_df['PB_Ratio'].between(pb_ratio_range[0], pb_ratio_range[1], inclusive='both')]
    if roe_range:
        filtered_df = filtered_df[filtered_df['ROE'].between(roe_range[0], roe_range[1], inclusive='both')]
    if debt_to_equity_range:
        filtered_df = filtered_df[filtered_df['Debt_to_Equity'].between(debt_to_equity_range[0], debt_to_equity_range[1], inclusive='both')]
    if free_cash_flow_range:
        filtered_df = filtered_df[filtered_df['Free_Cash_Flow'].between(free_cash_flow_range[0], free_cash_flow_range[1], inclusive='both')]
    if roic_range:
        filtered_df = filtered_df[filtered_df['ROIC'].between(roic_range[0], roic_range[1], inclusive='both')]
    if ps_ratio_range:
        filtered_df = filtered_df[filtered_df['PS_Ratio'].between(ps_ratio_range[0], ps_ratio_range[1], inclusive='both')]
    if ev_to_ebitda_range:
        filtered_df = filtered_df[filtered_df['EV_to_EBITDA'].between(ev_to_ebitda_range[0], ev_to_ebitda_range[1], inclusive='both')]

    return filtered_df

# Get ticker lists
sp500_tickers = get_sp500_tickers()
ftse100_tickers = get_ftse100_tickers()
euro_stoxx600_tickers = get_euro_stoxx600_tickers()

# Validate tickers
sp500_tickers = validate_tickers(sp500_tickers)
ftse100_tickers = validate_tickers(ftse100_tickers)
euro_stoxx600_tickers = validate_tickers(euro_stoxx600_tickers)

# Print number of tickers fetched and first few tickers for verification
print(f"S&P 500 Tickers ({len(sp500_tickers)}): {sp500_tickers[:5]} ...")
print(f"FTSE 100 Tickers ({len(ftse100_tickers)}): {ftse100_tickers[:5]} ...")
print(f"Euro Stoxx 600 Tickers ({len(euro_stoxx600_tickers)}): {euro_stoxx600_tickers[:5]} ...")

# Calculate financial ratios
ratios = calculate_ratios(sp500_tickers, ftse100_tickers, euro_stoxx600_tickers)

# Print number of tickers processed and first few tickers for verification
print(f"Total Tickers Processed: {len(ratios)}")
print(ratios.head())

# Example of filtering companies with multiple criteria
filtered_companies = filter_companies(
    ratios,
    pe_ratio_range=(10, 20),
    pb_ratio_range=(1, 3),
    roe_range=(0.05, 0.5),
    debt_to_equity_range=(0, 100),
    free_cash_flow_range=(1e9, 1e11),
    roic_range=(0.01, 0.15),
    ps_ratio_range=(1, 5),
    ev_to_ebitda_range=(5, 20)
)

# Display filtered companies
print("Filtered Companies based on multiple criteria:")
print(filtered_companies)

# Check if any FTSE 100 and Euro Stoxx 600 tickers are present in the final DataFrame
#ftse_in_ratios = [ticker for ticker in ftse100_tickers if ticker in ratios['Ticker'].values]
#euro_stoxx_in_ratios = [ticker for ticker in euro_stoxx600_tickers if ticker in ratios['Ticker'].values]
#print(f"FTSE 100 Tickers in Final DataFrame: {len(ftse_in_ratios)}")
#print(ftse_in_ratios)
#print(f"Euro Stoxx 600 Tickers in Final DataFrame: {len(euro_stoxx_in_ratios)}")
#print(euro_stoxx_in_ratios)

# Display all companies without applying any filters
#pd.set_option('display.max_columns', None)  # Ensure all columns are shown
#pd.set_option('display.width', None)        # Ensure the output fits the width of the terminal
#print(ratios.to_string(index=False))

S&P 500 Tickers (501): ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN'] ...
FTSE 100 Tickers (99): ['III.L', 'ADM.L', 'AAF.L', 'AAL.L', 'ANTO.L'] ...
Euro Stoxx 600 Tickers (9): ['NL', 'CH', 'GB', 'CH', 'SHELL'] ...


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SHELL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SHELL&crumb=e9CL4tIZE9S


Total Tickers Processed: 609
  Ticker   PE_Ratio   PB_Ratio      ROE  Debt_to_Equity  Free_Cash_Flow     ROIC  PS_Ratio  \
0    MMM  15.928954  17.520063  0.24314         345.812    1.080262e+10  0.07516  2.103544   
1    AOS  18.569122   6.148623  0.30342           8.637    3.881000e+08  0.14533  2.991739   
2    ABT  21.381323   4.863262  0.14441          37.525    5.387750e+09  0.06255  4.695273   
3   ABBV  15.737583  41.920620      NaN         924.059             NaN      NaN  6.105434   
4    ACN  24.761940   7.141792  0.26000          16.078    8.045156e+09  0.12244  3.071986   

   Enterprise_Value  EV_to_EBITDA  
0      7.100210e+10         9.660  
1      1.163881e+10        14.041  
2      1.991524e+11        18.840  
3      3.874479e+11        14.760  
4      1.991375e+11        17.654  
Filtered Companies based on multiple criteria:
    Ticker   PE_Ratio  PB_Ratio      ROE  Debt_to_Equity  Free_Cash_Flow     ROIC  PS_Ratio  \
8      AFL  14.223088  2.164163  0.23538        